In [16]:
import os
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Download dataset 
https://drive.google.com/drive/folders/1vI8Bkk5DojitLNkpz-UT30jOEay0XXon?usp=sharing

## Load data

In [17]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
"""path = '/content/drive/MyDrive/hw5/'
x_train = np.load(path+"x_train.npy")
y_train = np.load(path+"y_train.npy")
x_test = np.load(path+"x_test.npy")
y_test = np.load(path+"y_test.npy")"""

x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")
x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [19]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [20]:
# transform format
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)

In [21]:
# train test (validation) split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2022)

In [22]:
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

datagen.fit(x_train)

## Build model & training (Keras)

In [23]:
# build model with pretrained model (resnet50)
model = keras.models.Sequential()
model.add(Input(shape=(32, 32, 3), name='input'))
model.add(ResNet50(weights='imagenet', include_top=False, pooling='max'))
model.add(Flatten())
model.add(Dense(512, activation='relu', name='fc1'))
model.add(Dense(512, activation='relu', name='fc2'))
model.add(Dense(10, activation='softmax', name='prediction'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 fc1 (Dense)                 (None, 512)               1049088   
                                                                 
 fc2 (Dense)                 (None, 512)               262656    
                                                                 
 prediction (Dense)          (None, 10)                5130      
                                                                 
Total params: 24,904,586
Trainable params: 24,851,466
Non-trainable params: 53,120
_________________________________________________________________


In [24]:
# initiate SGD optimizer
opt = tf.keras.optimizers.SGD()
# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [25]:
# Setup some hyperparameters
batch_size = 64
epochs = 15 # 20

# Fit the data into model
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                  epochs=epochs, verbose=1, workers=4,
                  validation_data=(x_val, y_val))

Epoch 1/15
625/625 [==============================] - 46s 66ms/step - loss: 1.3922 - accuracy: 0.5139 - val_loss: 3.6017 - val_accuracy: 0.1536
Epoch 2/15
625/625 [==============================] - 38s 60ms/step - loss: 0.9210 - accuracy: 0.6823 - val_loss: 0.8764 - val_accuracy: 0.6961
Epoch 3/15
625/625 [==============================] - 37s 60ms/step - loss: 0.7670 - accuracy: 0.7355 - val_loss: 0.7532 - val_accuracy: 0.7487
Epoch 4/15
625/625 [==============================] - 39s 62ms/step - loss: 0.6778 - accuracy: 0.7653 - val_loss: 0.6899 - val_accuracy: 0.7651
Epoch 5/15
625/625 [==============================] - 37s 60ms/step - loss: 0.6117 - accuracy: 0.7883 - val_loss: 0.6259 - val_accuracy: 0.7865
Epoch 6/15
625/625 [==============================] - 40s 64ms/step - loss: 0.5620 - accuracy: 0.8059 - val_loss: 0.6338 - val_accuracy: 0.7851
Epoch 7/15
625/625 [==============================] - 39s 62ms/step - loss: 0.5205 - accuracy: 0.8181 - val_loss: 0.6265 - val_accuracy:

In [26]:
model.save('model.h5')

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [27]:
y_pred = model.predict(x_test) # 10000 samples, each sample with probaility of 10 classes
y_pred = np.argmax(y_pred, axis=1) # argmax to find the predict class with highest probability.

In [28]:
assert y_pred.shape == (10000,)

In [29]:
#y_test = np.load(path+"y_test.npy")
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.8242
